<a href="https://colab.research.google.com/github/JouJoi/Hugo/blob/main/TESIS_Extrac_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extractor de Corpus de comentarios directos a publicaciones de Reddit
Se pone en un dataframe que luego se exporta como GS los siguientes datos:

*   Numero de upvotes
*   Numero de downvotes (un desproposito)
*   Comentarios directos a la PO
*   Nombre de usuario ligado al comentario
*   fecha de publicación (en lo que sospecho es un formato raro)
*   Numero de palabras del comentario

Esto ya funciona y muy bien, las únicas partes que tengo que editar para usarlo con hilos diferentes en un futuro es la lista de los urls y el nombre del doc nuevo de GS que se abre desde aquí (para que se abra uno diferente).

In [ ]:

# Montar Google Drive en Colab para guardar y acceder a archivos fácilmente
from google.colab import drive
drive.mount('/content/drive')

# Instalar librerías necesarias
!pip install praw gspread pandas google-auth gspread_dataframe

# Importar librerías
import praw
import pandas as pd
import gspread
from google.auth import default
from google.colab import auth
import gspread_dataframe as gd

# Autenticación de usuario en Google Colab
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [ ]:
# Conectar a Google Sheets
sheet_name = "Reddit_Data1"
spreadsheet = gc.create(sheet_name)  # Crear nuevo documento de Google Sheets
worksheet = spreadsheet.sheet1  # Acceder a la primera hoja

In [ ]:
# Configuración de Reddit API con credenciales
reddit = praw.Reddit(
    client_id='3Cea377PCzoMDxXuNy0APA',
    client_secret='LEQPne5K9twDv-Cr6u79fUf--yPFvA',
    user_agent='Data_Extractor'
)

La cosa entre estas notas de texto es para corregir el formato de la fecha



In [ ]:
from datetime import datetime

# Función para extraer información de un hilo de Reddit
def extraer_datos_hilo(url):
    submission = reddit.submission(url=url)

    data = {
        'upvotes': [],
        'downvotes': [],
        'comentario': [],
        'usuario': [],
        'fecha': [],
        'numero_palabras': []
    }

    # Extraer comentarios directos al post
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        data['upvotes'].append(comment.ups)
        data['downvotes'].append(comment.downs)
        data['comentario'].append(comment.body)
        data['usuario'].append(comment.author.name if comment.author else 'Unknown')

        # Convertir timestamp a fecha legible
        fecha_legible = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        data['fecha'].append(fecha_legible)

        data['numero_palabras'].append(len(comment.body.split()))

    # Convertir a DataFrame
    df = pd.DataFrame(data)
    return df


La cosa entre estas notas de texto es para corregir el formato de la fecha, el siguiente cuadro de código es la versión sin corregir el formato, que no borro para recordar la diferencia.


In [ ]:
# Función para extraer información de un hilo de Reddit
#def extraer_datos_hilo(url):
#    submission = reddit.submission(url=url)
#
#    data = {
#        'upvotes': [],
#        'downvotes': [],
#        'comentario': [],
#        'usuario': [],
#        'fecha': [],
#        'numero_palabras': []
#    }
#
    # Extraer comentarios directos al post
#    submission.comments.replace_more(limit=None)
#    for comment in submission.comments.list():
#        data['upvotes'].append(comment.ups)
#        data['downvotes'].append(comment.downs)
#        data['comentario'].append(comment.body)
#        data['usuario'].append(comment.author.name if comment.author else 'Unknown')
#        data['fecha'].append(comment.created_utc)
#        data['numero_palabras'].append(len(comment.body.split()))

    # Convertir a DataFrame
#    df = pd.DataFrame(data)
#    return df


In [ ]:
hilos = [
    "https://www.reddit.com/r/SoyUnIdiota/comments/13enq6t/soy_culero_por_cobrarle_a_lo_chino_a_un_familiar/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/13dhkok/soy_culero_por_enojarme_con_mi_novia_por_robar/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/19213oa/sc_por_denunciar_a_mis_ex_amigos_por_agresión/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/11qxvee/sc_por_dar_a_mi_hija_prácticamente_en_adopción/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/12a6kff/sc_por_dejar_a_mi_amiga_en_la_discoteca/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/11qvd3z/sc_por_no_querer_trabajar_gratis_para_mi_familia/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/1clqapl/sc_por_no_querer_darle_pension_a_mi_exesposa/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/11pqk2y/sc_por_no_contratar_gente_con_pronombres/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/13vqtrw/soy_culero_por_no_querer_ser_aval_de_un_amigo/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button",
    "https://www.reddit.com/r/SoyUnIdiota/comments/18ibzu9/sc_por_estafar_a_mi_papá_por_andar_de_culo_suelto/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button"
]

In [ ]:
# Extraer datos de cada hilo y escribirlos en el Google Sheet
for idx, url in enumerate(hilos, start=1):
    df = extraer_datos_hilo(url)

    # Crear nueva hoja en el Google Sheet para cada hilo
    worksheet_name = f"Hilo_{idx}"
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="20")

    # Subir DataFrame a la hoja
    gd.set_with_dataframe(worksheet, df)

print(f"Datos extraídos y almacenados en el documento de Google Sheets: {sheet_name}")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Datos extraídos y almacenados en el documento de Google Sheets: Reddit_Data1


Hasta aquí el chatGPT usado es el de Extracción de datos Reddit del 28 de Septiembre del 2024
